<center><b><font size=10>Expert Systems - BuildWise Expert System</font></b></center>
<center><b><font color='red' size=5>Project Prototype</font></b></center>

---
#### Background on the Project
This notebook showcases the development and testing of the **BuildWise Expert System**, a prototype expert system designed to aid in building assessments, particularly in post-disaster scenarios. By leveraging fuzzy logic, the system provides structured evaluations of building damage, prioritization of reconstruction efforts, and efficient resource allocation. This project addresses real-world challenges such as incomplete data, conflicting reports, and subjective decision-making.

#### What is in This Notebook
This notebook is structured to include:
1. **Code Implementation**: The `BuildWiseExpertSystem` class is implemented to encapsulate the functionality of the system, including initialization, fuzzy logic rules, and forward and backward chaining techniques.
2. **Configuration Loading**: Membership functions, rules, and dependencies are loaded from JSON configuration files to ensure modularity and adaptability.
3. **Forward Chaining Tests**: Several scenarios are tested to evaluate how the system processes input data to generate outputs like damage levels, priority levels, and risk levels.
4. **Backward Chaining Tests**: Goals such as classifying damage as severe or prioritizing high-risk buildings are evaluated to validate the system's reasoning capabilities.
5. **Insights and Outputs**: The results from forward and backward chaining tests are analyzed to demonstrate the system's performance and highlight areas for improvement.

This notebook serves as a practical demonstration of using fuzzy logic and rule-based systems to address complex real-world challenges effectively.

---
#### Importing Required Libraries
This cell imports the necessary libraries and modules required for implementing the BuildWise Expert System:
- `numpy` for numerical computations.
- `skfuzzy` and `skfuzzy.control` for fuzzy logic operations and control systems.
- `json` for reading configuration files containing rules and membership functions.


In [1]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import json

---
#### BuildWiseExpertSystem Class Implementation
This cell defines the `BuildWiseExpertSystem` class, which encapsulates the logic for initializing and running the expert system. Key features include:
- **Initialization**: Loads membership configurations, rules, and dependencies from configuration files.
- **Input Variables**: Defines various antecedents like crack width, debris load, and structural tilt.
- **Output Variables**: Defines consequents such as damage level, priority level, and risk level.
- **Membership Functions**: Configures fuzzy membership functions for all variables.
- **Rule Definition**: Sets up fuzzy rules for evaluation.
- **Adjustive Rules**: Implements rules to fine-tune outputs based on additional conditions.
- **Forward Chaining**: Evaluates outputs based on provided inputs.
- **Backward Chaining**: Determines the conditions required to achieve specific goals.


In [2]:
class BuildWiseExpertSystem:
    def __init__(self, membership_configs, rules_config, adjustive_rules_config, dependency_map):
        """
         Initialize the system with the configs provided,
         To next,initialize the variables of the system,
         And define the rules of the system,
         As well as the adjustive rules for the system,
         Finally, create a simulation object of the system.
        """
        print("Initializing BuildWise Expert System...")
        self.membership_configs = membership_configs
        self.rules_config = rules_config
        self.dependency_map = dependency_map
        self.adjustive_rules_config = adjustive_rules_config
        print("="*30)
        self._initialize_variables()
        print("="*30)
        self._define_rules()
        print("="*30)
        self._create_system()
        print("="*30)
        print("BuildWise Expert System initialized successfully.")
        print("-"*10)
        
    def _initialize_inputs(self):
        """Inputs."""
        print("-"*10)
        print("Initializing input variables...")
        self.crack_width = ctrl.Antecedent(np.arange(0, 11, 1), 'crack_width')
        self.foundation_stability = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'foundation_stability')
        self.structural_tilt = ctrl.Antecedent(np.arange(0, 15, 1), 'structural_tilt')
        self.prob_structural_tilt = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'prob_structural_tilt')
        self.debris_load = ctrl.Antecedent(np.arange(0, 101, 5), 'debris_load')
        self.prob_foundational_damage = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'prob_foundational_damage')
        self.partial_wall_collapse = ctrl.Antecedent([0, 1], 'partial_wall_collapse')
        self.prob_structural_failure = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'prob_structural_failure')
        self.damage_probability = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'damage_probability')
        self.num_adj_buildings = ctrl.Antecedent(np.arange(0, 5, 1), 'num_adj_buildings')
        self.env_impact = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'env_impact')
        
        self.building_type = ctrl.Antecedent(np.arange(0, 3, 1), 'building_type') # 0=Residential, 1=School, 2=Hospital
        self.pop_density = ctrl.Antecedent(np.arange(0, 50001, 500), 'pop_density')
        self.prob_inaccessibility = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'prob_inaccessibility')
        self.prob_UXO = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'prob_UXO')
        self.time_since_damage = ctrl.Antecedent(np.arange(0, 13, 1), 'time_since_damage')
        self.evacuation_routes = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'evacuation_routes')
        self.school_proximity = ctrl.Antecedent(np.arange(0, 1001, 100), 'school_proximity')
        self.hazard_probability = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'hazard_probability')
        
        self.uxo_confidence = ctrl.Antecedent(np.arange(0, 101, 5), 'uxo_confidence')
        self.blockage_prob = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'blockage_prob')
        self.debris_distance = ctrl.Antecedent(np.arange(0, 101, 5), 'debris_distance')
        self.fragility_index = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'fragility_index')
        self.safe_zone_distance = ctrl.Antecedent(np.arange(0, 501, 10), 'safe_zone_distance')
        print("Input variables initialized.")
        print("-"*10)
        
    def _initialize_outputs(self):
        """Outputs."""
        print("Initializing output variables...")
        self.damage_level = ctrl.Consequent(np.arange(0, 4, 1), 'damage_level')
        self.priority_level = ctrl.Consequent(np.arange(1, 4, 1), 'priority_level')
        self.risk_level = ctrl.Consequent(np.arange(0, 4, 1), 'risk_level')
        self.resource_allocation = ctrl.Consequent(np.arange(0, 101, 5), 'resource_allocation')
        print("Output variables initialized.")
        print("-"*10)

    def _add_membership_functions(self, variable_name, configs):
        """Helper to add membership functions to a variable."""
        print(f"Adding membership functions for: {variable_name}")
        variable = getattr(self, variable_name)
        for label, params in configs.items():
            print(f"    Adding membership function '{label}' with params {params}.")
            variable[label] = fuzz.trimf(variable.universe, params)
            
    def _define_membership_functions(self):
        """Define membership functions for all variables."""
        print("Defining membership functions for all variables...")
        print("_"*15)
        for variable_name, configs in self.membership_configs.items():
            self._add_membership_functions(variable_name, configs)
        print("_"*15)
        print("Membership functions defined.")
        print("-"*10)
            
    def _initialize_variables(self):
        print("Initializing variables...")
        self._initialize_inputs()
        self._initialize_outputs()
        self._define_membership_functions()
        print("Variables initialized.")

    def _add_rules(self, rules, category_name):
        print(f"Adding rules for category: {category_name}...")
        for rule in rules:
            condition = eval(rule["conditions"])
            action = eval(rule["action"])
            self.rules.append(ctrl.Rule(condition, action))
        print(f"    Added {len(rules)} rules to category: {category_name}.")
    def _define_rules(self):
        print("Defining rules...")
        print("-"*10)
        self.rules = []
        # Load rules from a structured configuration
        for category_name, rule_set in self.rules_config.items():
            self._add_rules(rule_set, category_name)
        print("-"*10)
        print(f"Total rules defined: {len(self.rules)}")
            
    def _apply_adjustive_rules(self, rules, inputs_dict):
        for rule in rules:
            if eval(rule["condition"]):
                print("-"*10)
                print(f"Condition met: {rule['condition']}")
                for action in rule["actions"]:  # Iterate through each action
                    print(f"    Executing action: {action}")
                    exec(action)

    
    def _add_damage_adjustive_rules(self, inputs_dict):
        print("_"*15)
        print("Applying damage adjustive rules...")
        self._apply_adjustive_rules(self.adjustive_rules_config["damage"], inputs_dict)
    
    def _add_priority_adjustive_rules(self, inputs_dict):
        print("_"*15)
        print("Applying priority adjustive rules...")
        self._apply_adjustive_rules(self.adjustive_rules_config["priority"], inputs_dict)
    
    def _add_risk_adjustive_rules(self, inputs_dict):
        print("_"*15)
        print("Applying risk adjustive rules...")
        self._apply_adjustive_rules(self.adjustive_rules_config["risk"], inputs_dict)
    
    def _adjustive_rules(self, inputs_dict):
        print("Applying all adjustive rules...")
        self._add_damage_adjustive_rules(inputs_dict)
        self._add_priority_adjustive_rules(inputs_dict)
        self._add_risk_adjustive_rules(inputs_dict)
        print("_"*15)
        print("Adjustive rules applied.")

    def _get_output_label(self, consequent, crisp_value, output_name):
        """
        Get the label corresponding to the highest membership degree
        for a crisp value from a fuzzy variable.
        """
        membership_degrees = {}
        for label, mf in consequent.terms.items():
            degree = fuzz.interp_membership(consequent.universe, mf.mf, crisp_value)
            membership_degrees[label] = degree
        max_label = max(membership_degrees, key=membership_degrees.get)
        print(f"Crisp value {crisp_value:.2f} for '{output_name}' maps to label '{max_label}' with degree {membership_degrees[max_label]:.2f}")
        return max_label
        
    def calculate_probabilities(self, crack_width_input, tilt_input, debris_load_input):
        print("Calculating probabilities...")
        # Calculate individual probabilities
        PC = fuzz.interp_membership(self.crack_width.universe, self.crack_width['large'].mf, crack_width_input)
        PT = fuzz.interp_membership(self.structural_tilt.universe, self.structural_tilt['high'].mf, tilt_input)
        PD = fuzz.interp_membership(self.debris_load.universe, self.debris_load['high'].mf, debris_load_input)
        
        # Calculate PIrreparable
        PIrreparable = PC * PT * PD
        print(f"PC: {PC:.2f}, PT: {PT:.2f}, PD: {PD:.2f}, PIrreparable: {PIrreparable:.2f}")
        return PC, PT, PD, PIrreparable    
        
    def evaluate_condition(self, condition_str, inputs):
        """
        Evaluate a condition string with logical operators (| for OR, & for AND).
        Returns:
            - A tuple (condition_satisfied, term_details), where:
                - condition_satisfied: True if the condition is satisfied, False otherwise.
                - term_details: A list of dictionaries containing details about each term's evaluation.
        """
        # Split the condition into terms and operators
        terms = []
        operators = []
        for token in condition_str.split():
            if token in ["|", "&"]:
                operators.append(token)
            else:
                terms.append(token)
    
        # Evaluate each term and store details
        term_details = []
        for term in terms:
            if "['" in term:
                # Extract variable name and label
                variable_name = term.split("self.")[1].split("[")[0]
                label = term.split("['")[1].split("']")[0]
    
                # Get the variable and its membership function
                variable = getattr(self, variable_name)
                value = inputs[variable_name]
    
                # Calculate membership degree
                membership_degree = fuzz.interp_membership(variable.universe, variable[label].mf, value)
                term_satisfied = membership_degree >= 0.5  # Threshold for satisfaction
    
                # Store term details
                term_details.append({
                    "term": term,
                    "value": value,
                    "label": label,
                    "membership_degree": membership_degree,
                    "satisfied": term_satisfied
                })
            else:
                raise ValueError(f"Invalid term in condition: {term}")
    
        # Combine results using logical operators
        if not operators:
            return term_details[0]["satisfied"], term_details  # Single term
    
        result = term_details[0]["satisfied"]
        for i, operator in enumerate(operators):
            if operator == "&":
                result = result and term_details[i + 1]["satisfied"]
            elif operator == "|":
                result = result or term_details[i + 1]["satisfied"]
            else:
                raise ValueError(f"Invalid operator: {operator}")
    
        return result, term_details
        
    def evaluate_system_backward(self, goal):
        """
        Perform backward chaining to achieve a specific goal.
        """
        print(f"Starting backward chaining for goal: {goal}")
        if goal not in self.dependency_map:
            print(f"Error: Goal '{goal}' not found in dependency map.")
            return False
    
        # Get the conditions and inputs required for the goal
        conditions_list = self.dependency_map[goal]
        inputs_required = set()
    
        # Collect all required inputs
        for condition in conditions_list:
            inputs_required.update(condition["inputs"])
    
        print(f"Inputs required for goal '{goal}': {inputs_required}")
    
        # Prompt the user for required inputs
        inputs = {}
        for input_name in inputs_required:
            # Retrieve the variable and its universe of discourse
            variable = getattr(self, input_name)
            min_value = variable.universe[0]
            max_value = variable.universe[-1]
    
            # Prompt the user with the valid range
            while True:
                try:
                    value = float(input(
                        f"Enter value for '{input_name}' (range: {min_value} to {max_value}): "
                    ))
                    if min_value <= value <= max_value:
                        inputs[input_name] = value
                        break
                    else:
                        print(f"Error: Value must be between {min_value} and {max_value}. Please try again.")
                except ValueError:
                    print("Error: Invalid input. Please enter a numeric value.")
    
        # Evaluate all conditions for the goal
        goal_achieved = False
        for condition in conditions_list:
            condition_str = condition["conditions"]
            print('-'*10)
            print(f"Evaluating condition: {condition_str}")
    
            # Evaluate the condition using the helper method
            condition_satisfied, term_details = self.evaluate_condition(condition_str, inputs)
    
            # Print term details
            for term in term_details:
                print(
                    f"    Term: {term['term']}, "
                    f"Value: {term['value']}, "
                    f"Label: {term['label']}, "
                    f"Membership Degree: {term['membership_degree']:.2f}, "
                    f"Satisfied: {term['satisfied']}"
                )
    
            if condition_satisfied:
                print(f"Condition '{condition_str}' is satisfied.")
                goal_achieved = True
            else:
                print(f"Condition '{condition_str}' is not satisfied.")

        print('='*30)
        if goal_achieved:
            print(f"Goal '{goal}' achieved!")
        else:
            print(f"Goal '{goal}' could not be achieved with the provided inputs.")
    
        return goal_achieved
        
    def _create_system(self):
        # Create the system
        print("Creating the system...")
        self.system_ctrl = ctrl.ControlSystem(self.rules)
        self.simulation = ctrl.ControlSystemSimulation(self.system_ctrl)
        print("System created.")

    def evaluate_system_forward(self, inputs):
        print("=" * 30)
        print("Evaluating system...")
        for key, value in inputs.items():
            print(f"    Setting input: {key} = {value}")
            self.simulation.input[key] = value
    
        inputs_dict = self.simulation._get_inputs()
    
        # Calculate probabilities and add to simulation input
        PC, PT, PD, PIrreparable = self.calculate_probabilities(
            inputs_dict["crack_width"], inputs_dict["structural_tilt"], inputs_dict["debris_load"]
        )
        self.simulation.input['damage_probability'] = PIrreparable
        self.simulation.compute()
    
        print("=" * 30)
    
        # Initialize outputs with default values if not set by rules
        if "damage_level" not in self.simulation.output:
            print("Warning: No rules were triggered for 'damage_level'. Assigning default value: 0 (minor).")
            self.simulation.output["damage_level"] = 0  # Default to 'minor'
        if "priority_level" not in self.simulation.output:
            print("Warning: No rules were triggered for 'priority_level'. Assigning default value: 1 (low).")
            self.simulation.output["priority_level"] = 1  # Default to 'low'
        if "risk_level" not in self.simulation.output:
            print("Warning: No rules were triggered for 'risk_level'. Assigning default value: 0 (low).")
            self.simulation.output["risk_level"] = 0  # Default to 'low'
        if "resource_allocation" not in self.simulation.output:
            print("Warning: No rules were triggered for 'resource_allocation'. Assigning default value: 0%.")
            self.simulation.output["resource_allocation"] = 0  # Default to 0%
    
        # Apply adjustive rules
        self._adjustive_rules(inputs_dict)
        print("=" * 30)
    
        # Initialize outputs dictionary
        outputs = {}
        result = {}
    
        # Check and handle each output
        for output_name, consequent in {
            "damage_level": self.damage_level,
            "priority_level": self.priority_level,
            "risk_level": self.risk_level,
        }.items():
            crisp_value = self.simulation.output[output_name]
            label = self._get_output_label(consequent, crisp_value, output_name.capitalize())
            outputs[output_name] = label
            result[output_name] = crisp_value
    
        # Handle resource allocation separately
        result["resource_allocation"] = self.simulation.output["resource_allocation"]
    
        print("Evaluation Results:")
        print(f"    Damage Level: {outputs['damage_level']}")
        print(f"    Priority Level: {outputs['priority_level']}")
        print(f"    Risk Level: {outputs['risk_level']}")
        print(f"    Resource Allocation: {result['resource_allocation']}%")
    
        return result


---
#### System Initialization
This cell initializes the BuildWise Expert System by loading configurations from JSON files:
- **Membership Configurations**: Defines membership functions for each variable.
- **Rules Configuration**: Specifies rules for fuzzy inference.
- **Adjustive Rules Configuration**: Includes additional conditions for fine-tuning outputs.
- **Dependency Map**: Lists dependencies for backward chaining.
The system is then instantiated using these configurations, preparing it for evaluation.


In [3]:
with open("membership_configs.json", "r") as file:
    MEMBERSHIP_CONFIGS = json.load(file)

with open("rules_config.json", "r") as file:
    RULES_CONFIG = json.load(file)

with open("adjustive_rules_config.json", "r") as file:
    ADJUSTIVE_RULES_CONFIG = json.load(file)

with open('dependency_map.json', "r") as file:
        DEPENDENCY_MAP = json.load(file)
# Initialize the system
system = BuildWiseExpertSystem(MEMBERSHIP_CONFIGS, RULES_CONFIG, ADJUSTIVE_RULES_CONFIG, DEPENDENCY_MAP)

Initializing BuildWise Expert System...
Initializing variables...
----------
Initializing input variables...
Input variables initialized.
----------
Initializing output variables...
Output variables initialized.
----------
Defining membership functions for all variables...
_______________
Adding membership functions for: crack_width
    Adding membership function 'small' with params [0, 0, 5].
    Adding membership function 'medium' with params [3, 5, 8].
    Adding membership function 'large' with params [6, 10, 10].
Adding membership functions for: foundation_stability
    Adding membership function 'low' with params [0, 0, 0.5].
    Adding membership function 'medium' with params [0.3, 0.5, 0.7].
    Adding membership function 'high' with params [0.6, 1, 1].
Adding membership functions for: prob_foundational_damage
    Adding membership function 'low' with params [0, 0, 0.5].
    Adding membership function 'medium' with params [0.3, 0.5, 0.7].
    Adding membership function 'high' w

---
#### Defining Test Cases for Forward Chaining
This cell defines input scenarios for forward chaining tests:
1. **Minor Damage Scenario**: Inputs representing minor structural damage.
2. **Severe Damage Scenario**: Inputs representing severe structural damage.
3. **Moderate Damage Scenario**: Inputs representing moderate structural damage.
These test cases provide a comprehensive evaluation of the system's ability to classify damage levels and assign priorities.


In [4]:
# Forward Chaining Test 1: Minor Damage Scenario
inputs_minor = {
    "crack_width": 2,  # Small crack
    "foundation_stability": 0.8,  # High stability
    "structural_tilt": 3,  # Low tilt
    "prob_structural_tilt": 0.1,  # Low probability
    "debris_load": 20,  # Low debris load
    "prob_foundational_damage": 0.2,  # Low probability
    "partial_wall_collapse": 0,  # No collapse
    "prob_structural_failure": 0.1,  # Low probability
    "num_adj_buildings": 1,  # Low number of adjacent buildings
    "env_impact": 0.3,  # Low environmental impact
    "building_type": 0,  # Residential
    "pop_density": 1000,  # Low population density
    "prob_inaccessibility": 0.2,  # Low probability
    "prob_UXO": 0.1,  # Low probability
    "time_since_damage": 2,  # Recent damage
    "evacuation_routes": 0.8,  # Good evacuation routes
    "school_proximity": 500,  # Medium proximity
    "hazard_probability": 0.2,  # Low hazard probability
    "uxo_confidence": 20,  # Low confidence
    "blockage_prob": 0.2,  # Low blockage probability
    "debris_distance": 50,  # Medium debris distance
    "fragility_index": 0.3,  # Low fragility
    "safe_zone_distance": 200  # Medium safe zone distance
}

# Forward Chaining Test 2: Severe Damage Scenario
inputs_severe = {
    "crack_width": 8,  # Large crack
    "foundation_stability": 0.3,  # Low stability
    "structural_tilt": 10,  # High tilt
    "prob_structural_tilt": 0.8,  # High probability
    "debris_load": 80,  # High debris load
    "prob_foundational_damage": 0.7,  # High probability
    "partial_wall_collapse": 1,  # Partial collapse
    "prob_structural_failure": 0.8,  # High probability
    "num_adj_buildings": 3,  # High number of adjacent buildings
    "env_impact": 0.7,  # High environmental impact
    "building_type": 2,  # Hospital
    "pop_density": 15000,  # High population density
    "prob_inaccessibility": 0.7,  # High probability
    "prob_UXO": 0.8,  # High probability
    "time_since_damage": 10,  # Long time since damage
    "evacuation_routes": 0.2,  # Limited evacuation routes
    "school_proximity": 100,  # Close proximity to school
    "hazard_probability": 0.9,  # High hazard probability
    "uxo_confidence": 80,  # High confidence
    "blockage_prob": 0.8,  # High blockage probability
    "debris_distance": 10,  # Close debris distance
    "fragility_index": 0.8,  # High fragility
    "safe_zone_distance": 50  # Close safe zone distance
}

# Forward Chaining Test 3: Moderate Damage Scenario
inputs_moderate = {
    "crack_width": 5,  # Medium crack
    "foundation_stability": 0.5,  # Medium stability
    "structural_tilt": 7,  # Medium tilt
    "prob_structural_tilt": 0.5,  # Medium probability
    "debris_load": 50,  # Medium debris load
    "prob_foundational_damage": 0.5,  # Medium probability
    "partial_wall_collapse": 0,  # No collapse
    "prob_structural_failure": 0.5,  # Medium probability
    "num_adj_buildings": 2,  # Medium number of adjacent buildings
    "env_impact": 0.5,  # Medium environmental impact
    "building_type": 1,  # School
    "pop_density": 5000,  # Medium population density
    "prob_inaccessibility": 0.5,  # Medium probability
    "prob_UXO": 0.5,  # Medium probability
    "time_since_damage": 6,  # Medium time since damage
    "evacuation_routes": 0.5,  # Medium evacuation routes
    "school_proximity": 300,  # Medium proximity
    "hazard_probability": 0.5,  # Medium hazard probability
    "uxo_confidence": 50,  # Medium confidence
    "blockage_prob": 0.5,  # Medium blockage probability
    "debris_distance": 30,  # Medium debris distance
    "fragility_index": 0.5,  # Medium fragility
    "safe_zone_distance": 150  # Medium safe zone distance
}

---
#### Running Forward Chaining Tests
This cell runs the forward chaining tests defined earlier and evaluates the system's behavior for each scenario:
1. **Minor Damage Scenario**: Evaluates how the system processes inputs for minimal damage.
2. **Severe Damage Scenario**: Tests the system's response to severe damage inputs.
3. **Moderate Damage Scenario**: Validates the system's classification of moderate damage inputs.
Results include the classification of damage levels, priority levels, risk levels, and resource allocation.


In [5]:
# Run Forward Chaining Tests
print("=" * 30)
print("Forward Chaining Test 1: Minor Damage Scenario")
results_minor = system.evaluate_system_forward(inputs_minor)
# print(results_minor)

print("=" * 30)
print("Forward Chaining Test 2: Severe Damage Scenario")
results_severe = system.evaluate_system_forward(inputs_severe)
# print(results_severe)

print("=" * 30)
print("Forward Chaining Test 3: Moderate Damage Scenario")
results_moderate = system.evaluate_system_forward(inputs_moderate)
# print(results_moderate)

Forward Chaining Test 1: Minor Damage Scenario
Evaluating system...
    Setting input: crack_width = 2
    Setting input: foundation_stability = 0.8
    Setting input: structural_tilt = 3
    Setting input: prob_structural_tilt = 0.1
    Setting input: debris_load = 20
    Setting input: prob_foundational_damage = 0.2
    Setting input: partial_wall_collapse = 0
    Setting input: prob_structural_failure = 0.1
    Setting input: num_adj_buildings = 1
    Setting input: env_impact = 0.3
    Setting input: building_type = 0
    Setting input: pop_density = 1000
    Setting input: prob_inaccessibility = 0.2
    Setting input: prob_UXO = 0.1
    Setting input: time_since_damage = 2
    Setting input: evacuation_routes = 0.8
    Setting input: school_proximity = 500
    Setting input: hazard_probability = 0.2
    Setting input: uxo_confidence = 20
    Setting input: blockage_prob = 0.2
    Setting input: debris_distance = 50
    Setting input: fragility_index = 0.3
    Setting input: safe_z

---
#### Running Backward Chaining Tests
This cell performs backward chaining tests to determine the conditions required to achieve specific goals:
1. **Goal: Severe Damage**: Determines the input conditions needed for the system to classify damage as severe.
2. **Goal: High Priority**: Evaluates the conditions necessary for assigning high priority.
3. **Goal: High Risk**: Identifies the requirements for labeling a scenario as high risk.
These tests validate the system's reasoning capabilities and rule evaluation logic.


In [6]:
# Backward Chaining Test 1: Goal = damage_level['severe']
print("=" * 30)
print("Backward Chaining Test 1: Goal = damage_level['severe']")
goal_severe = "damage_level['severe']"
system.evaluate_system_backward(goal_severe)

# Backward Chaining Test 2: Goal = priority_level['high']
print("=" * 30)
print("Backward Chaining Test 2: Goal = priority_level['high']")
goal_high_priority = "priority_level['high']"
system.evaluate_system_backward(goal_high_priority)

# Backward Chaining Test 3: Goal = risk_level['high']
print("=" * 30)
print("Backward Chaining Test 3: Goal = risk_level['high']")
goal_high_risk = "risk_level['high']"
system.evaluate_system_backward(goal_high_risk)

Backward Chaining Test 1: Goal = damage_level['severe']
Starting backward chaining for goal: damage_level['severe']
Inputs required for goal 'damage_level['severe']': {'foundation_stability', 'damage_probability', 'structural_tilt'}
----------
Evaluating condition: self.structural_tilt['high'] | self.foundation_stability['low']
    Term: self.structural_tilt['high'], Value: 10.0, Label: high, Membership Degree: 0.29, Satisfied: False
    Term: self.foundation_stability['low'], Value: 0.25, Label: low, Membership Degree: 0.50, Satisfied: True
Condition 'self.structural_tilt['high'] | self.foundation_stability['low']' is satisfied.
----------
Evaluating condition: self.damage_probability['severe']
    Term: self.damage_probability['severe'], Value: 0.9, Label: severe, Membership Degree: 0.00, Satisfied: False
Condition 'self.damage_probability['severe']' is not satisfied.
Goal 'damage_level['severe']' achieved!
Backward Chaining Test 2: Goal = priority_level['high']
Starting backward cha

True